# Проект преобразования клиентских данных

### Цель проекта:

Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно осстановить персональную информацию, обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### Описание данных:

Набор данных находится в файле /datasets/insurance.csv.   

Признаки: 
+ пол
+ возраст 
+ зарплата застрахованного 
+ количество членов его семьи.
    
    
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

# Содержание проекта

[1. Загрузка данных](#1)  

+ [Вывод](#1_itog)  

[2.Умножение матриц](#2)  

+ [Вывод](#2_itog)  

[3.Алгоритм преобразования](#3)  

[4.Проверка алгоритма](#4)  

+ [Вывод](#3_itog)  

    
[Общий вывод](#itog)       

### Импорт необходимых библиотек:

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

## 1. Загрузка данных<a id="1"></a>

In [3]:
display(insurance.head(10))
display(insurance.info())
display(insurance.describe())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
# Проверим дубликаты
insurance.duplicated().sum()

153

In [5]:
# Удалим
insurance.drop_duplicates(inplace=True)

## Вывод<a id="1_itog"></a>

Оценили входные данные, удалили дубликаты,  необходимости в дальнейшей предварительной обработке нет.

## 2. Умножение матриц<a id="2"></a>

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Проверим предположение, что умножение матрицы признаков на произвольную обратимую матрицу не влияет на качество линейной регрессии.  

$$ a = XP((XP)^TXP)^{-1}(XP)^T)y $$  

Раскроем $ (XP)^T $ согласно свойству транспонирования, как $ P^TX^T $, получим  

$$ a = XP(P^TX^TXP)^{-1}P^TX^Ty $$ 
аналогично
$$ a = XP(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty $$ 
$$ a = XPP^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty $$ 
$ PP^{-1}$ является единичной матрицей Е. Заменим
$$ a = XE(X^TX)^{-1}EX^Ty $$
$$ a = X(X^TX)^{-1}X^Ty $$
$$ a = Xw $$

Что и требовалось доказать

## Вывод<a id="2_itog"></a>

Доказали, что умножение матрицы признаков на произвольную обратимую матрицу не влияет на результат линейной регрессии.

## 3. Алгоритм преобразования<a id="3"></a>

По результатам доказательства п. 2 в качестве алгоритма преобразования признаков примем умножение матрицы признаков на произвольную обратимую матрицу.  
Как мы уже доказали, это не повлияет на качество линейной регрессии.  
Далее , проверим результаты работы линейной регрессии по исходным признакам и по признакам, зашифрованным вышеописанным алгоритмом.

In [6]:
# Произвольная обратимая матрица
matrix = np.linalg.inv(np.random.normal(size=(4,4)))
display(matrix)

array([[ 1.29068777, -0.55175834,  1.72942906, -0.21972537],
       [ 5.05744057, -1.76396562,  7.43646142,  1.11600206],
       [-1.48722521,  0.09907094, -2.18643029, -0.16501082],
       [-4.80254981,  1.52351577, -6.25620478, -0.98600215]])

## 4. Проверка алгоритма<a id="4"></a>

In [7]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']
features_test = features @ matrix

In [8]:
display(features_test.head())

,0,1,2,3
0,-73562.527117,4842.567980,-108146.574218,-8139.986462
1,-56286.718189,3685.076962,-82748.529982,-6220.061180
2,-31085.063592,2029.334817,-45699.378699,-3432.863223
3,-61920.690023,4097.262110,-91030.489793,-6859.487279
4,-38673.678906,2535.808838,-56855.880208,-4275.754147


Обучим обе модели, на исходных и измененных данных:

In [9]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2 = r2_score(target, predictions)
print(r2)

0.4302010044852068


In [10]:
model = LinearRegression()
model.fit(features_test, target)
predictions = model.predict(features_test)
r2 = r2_score(target, predictions)
print(r2)

0.43020100448521104


## Вывод<a id="4_itog"></a>

Получили идентичные результаты метрики R2, что подтверждает верность теретических расчетов.

# Общий вывод<a id="itog"></a>

Согласно условиям проекта, разработан механизм шифрования данных клиента.  
Теоретически проверена и практически подтверждена корректность предложенного метода.  
